# ASL Translator
## This is the master preprocessing pipeline

### Plan for classification

    1. Train original images, output classification accuracy with original images
    2. Train preprocessed images, output classification accuracy with preprocessed images
    3. Compare accuracy between original images model and preprocessed images model

### Proposed pipeline

- preprocess the images (this stuff)
    - pick 2 or 3 different operation results
- feed into Kira's mask algorithm
    - removing background and keeping hand

- normal images for training -> preprocessed images for testing

In [2]:
import os
import glob
import imageio
import scipy.io
from scipy import ndimage
from skimage import feature
from skimage import exposure
from skimage import morphology
from skimage.color import rgb2gray, gray2rgb
from skimage.filters import gaussian
from skimage.filters.rank import median
from skimage.morphology import disk, ball
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [12]:
framesdir = "../image-classification-tensorflow-master/dataset/V/"
fnames = glob.glob(framesdir + "*.jpg")
fnames = [os.path.basename(name) for name in fnames]

random_indeces = np.random.choice(len(fnames), 1, replace=False)
chosen_fnames = np.take(fnames, random_indeces)
    
# original image
original_image = imageio.imread(framesdir + chosen_fnames[0])

# crop original image 5px from each side to remove border
height, width, channels = original_image.shape
original_image = original_image[5:width-5,5:height-5,:]

# grayscale image
gray_img = rgb2gray(original_image)

# enhance contrast of the gray image
high_contrast = exposure.equalize_hist(gray_img) * 255

# reduce noise from enhanced image
noiseless_img = median(high_contrast / 255, disk(3))

# perform canny edge detector on reduced noise image
edges_img = feature.canny(noiseless_img, sigma=1)

######## COMBINED IMAGES WITH EDGES_IMG ########

# overlay edges on original image
inv_edges = np.where(edges_img == 1, 0, 1)
combined_og = original_image * np.stack((inv_edges,inv_edges,inv_edges), axis=-1)   
for yy in range(combined_og.shape[0]):
    for xx in range(combined_og.shape[1]):
        if np.array_equal(combined_og[yy,xx], np.zeros(3)):
            combined_og[yy,xx] = np.array([255,255,0])

# now, the output of part 1 of preprocessing (combined_og) will be available in PART1OUTPUT
PART1OUTPUT = combined_og



# show this output if you like
# plt.imshow(PART1OUTPUT)




########### IGNORE BELOW THIS LINE -> Kira will implement this after midterm update ###########


# # Use threshold in gray image to find light and dark spots (separate thresholds may be used)
# threshold = 0.25
# light_spots = np.array((gray_img > 1- threshold).nonzero()).T
# plt.plot(light_spots[:, 1], light_spots[:, 0], 'o', color='red', markersize=1)
# dark_spots = np.array((gray_img < threshold).nonzero()).T
# plt.plot(dark_spots[:, 1], dark_spots[:, 0], 'o', color='black', markersize=1)

# bool_mask = np.zeros(gray_img.shape, dtype=np.bool)
# bool_mask[tuple(light_spots.T)] = True
# bool_mask[tuple(dark_spots.T)] = True
# seed_mask, num_seeds = ndimage.label(bool_mask)

# # Watershed mask to be used in the future to find the background and foreground
# ws = morphology.watershed(noiseless_img, seed_mask)
# # plt.imshow(ws)
# plt.show()

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [14]:
### ONLY USE THIS TO GENERATE THE IMAGES

selected_numberss = np.load('selected_numberss.npy')

framesdir = "../image-classification-tensorflow-master/dataset_og/"
savedirA = "../image-classification-tensorflow-master/datasetNormal800/"
savedirB = "../image-classification-tensorflow-master/datasetProcessed800/"

letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "space", "nothing", "del"]

# select 800 random numbers, between 2 and 2400
# selected_numbers = np.random.choice(2399, 800, replace=False) + 2
# print(selected_numbers)

for letter in letters:

    for i in selected_numbers:

        # original image
        original_image = imageio.imread(framesdir + letter + "/" + letter + str(i) + '.jpg')

        # crop original image 5px from each side to remove border
        height, width, channels = original_image.shape
        original_image = original_image[5:width-5,5:height-5,:]

        # save the image in the savedir
        imageio.imwrite(savedirA + "normal_" + letter + str(i) + '.jpg', original_image)
#         imageio.imwrite(savedirB + "processed_" + letter + str(i) + '.jpg', PART1OUTPUT)

In [13]:
# ### ONLY USE THIS TO GENERATE THE IMAGES
# import imageio
# import numpy as np
# import os
# import string

# source = '../image-classification-tensorflow-master/datasetProcessed800/'


# framesdir = "../image-classification-tensorflow-master/dataset_og/"
# savedirA = "../image-classification-tensorflow-master/datasetNormal800/"
# savedirB = "../image-classification-tensorflow-master/datasetProcessed800/"

# letters = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z", "space", "nothing", "del"]
# alphabet = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P", "Q", "R", "S", "T", "U", "V", "W", "X", "Y", "Z"]
# special = ["space", "nothing", "del"]

# selected_numberss = np.zeros(800)
# i = 0

# for filename in os.listdir(source): 
#     if filename.endswith('.jpg') and not filename == '.jpg':   
# #         print(filename)
#         if filename[10:15] == 'space':
#             temp = filename[15:]
#             selected_numberss[i] = temp[:len(temp) - 4]
#             i += 1
# print(selected_numberss)
# np.save('selected_numberss.npy', selected_numberss)
        


[1875. 2398. 2373.   59.  362. 1487. 2171. 2165. 1450.  955. 2005.  772.
 2011. 1530. 1256.  564.  570. 1041.  203. 1900. 1525.  983. 1531. 2010.
 1519.  773.  798.  968. 2158. 1323. 2170.  175.  820.  808.   64. 1135.
 1121. 2366. 1109. 2372. 1684. 1874. 1686. 2364.  413. 1137. 1651. 1889.
 2358.   72. 1123.  349. 1490.  822. 2199.  836. 1453. 2012. 2006. 1527.
  995. 1255. 1533.  981. 2210.  573. 2204.  201. 1057. 1042. 2205. 1903.
 1526. 1240. 1268. 2013. 1297. 1320.  638.  189.  348. 1136. 2359. 1888.
 2371. 1678. 2365. 1687. 1693. 1863. 1877. 1683.   88. 1654.   77. 2361.
  364.  370.  199. 2188. 1481.  833.  600. 2177.  947.  748.  984. 1250.
 1278. 1091. 1046. 1708.  563.  577. 2214. 1735. 2228. 1047. 1912. 1090.
  761. 1279. 2016. 1537.  991.  952. 1319.  167. 1443. 1325.  832.  826.
  365. 1127. 1899.   76. 1655.   89. 1696. 1858. 1680.  398. 1694. 1870.
   74. 1643.  429. 1657.   60.  415.   48. 2376. 1119. 1482.  830.  824.
 1441. 2148.  171. 1469.  944. 1284.  978. 1253.  9